In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Cell 3: Preprocess the data

# One-hot encode categorical columns
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

# Split into training and test sets
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

# Separate labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

# Now scale the entire dataset (all columns are numeric at this point)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

In [ ]:
# Split features and labels
#train_dataset = dataset.sample(frac=0.8, random_state=0)
#test_dataset = dataset.drop(train_dataset.index)

#train_labels = train_dataset.pop('expenses')
#test_labels = test_dataset.pop('expenses')
# Cell 4: Normalize numeric data
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#train_dataset = scaler.fit_transform(train_dataset)
#test_dataset = scaler.transform(test_dataset)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=[train_dataset.shape[1]]),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['mae']
  )
  return model

model = build_model()
model.summary()

In [ ]:
# Cell 6: Train the model

history = model.fit(
    train_dataset, train_labels,
    validation_split=0.1,
    epochs=200,
    callbacks=[tfdocs.modeling.EpochDots()],
    verbose=0
)

In [ ]:
# Cell 7: Plot training history (fixed)
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Model': history}, metric = "mae")
plt.ylabel('MAE')

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)